## **Soal 3 - Indonesia Covid-19 Maps 🌏**

Jumlah kasus infeksi virus corona Covid-19 masih terus mengalami peningkatan. Hingga Minggu (29/3/2020) sore, jumlah kasus infeksi Covid-19 yang telah dilaporkan di seluruh dunia adalah sebanyak __669.312 kasus__ dengan __145.609 pasien sembuh__ dan __31.700 meninggal dunia__. Wabah ini telah dilaporkan di lebih dari 170 negara di dunia, termasuk Indonesia.

Disediakan:
- Data sebaran Covid-19 tiap provinsi di Indonesia: [klik sini](https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Indonesia), untuk mengaksesnya silakan gunakan teknik _web scraping_.
- Data lokasi latitude & longitude tiap provinsi di Indonesia: [klik sini](https://raw.githubusercontent.com/LintangWisesa/Indonesia-Covid19-Maps/master/data/gps_indonesia.json), untuk mengaksesnya silakan lakukan GET request ke ```https://raw.githubusercontent.com/LintangWisesa/Indonesia-Covid19-Maps/master/data/gps_indonesia.json```.

Dengan data tersebut, buatlah sebuah peta sebaran Covid-19 per provinsi di Indonesia, dengan minimum requirements sebagai berikut:
- Tampilkan _marker_ di setiap provinsi.
- Saat _marker_ diklik akan menampilkan _popup message_ total kasus, pasien sembuh & kematian akibat Covid-19 di provinsi tersebut.
- Jawaban yang diminta berupa 1 file _notebook_ (__.ipynb__) berisi proses ekstraksi & data cleaning, beserta 1 file _html_ peta sebaran Covid-19 per provinsi (__.html__). 
- Contoh output dapat Anda simak di [bit.ly/cov19id](http://bit.ly/cov19id). Tampilan tidak harus sama, utamakan fitur.

✅ _Push & commit jawaban Anda ke sebuah repo github dengan nama __Soal3_Covid19__. Kirim & lampirkan url repo github Anda via email ke lintang@purwadhika.com_ dengan subject email "_NamaLengkap_Covid19_".

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import folium as fo
import plotly.graph_objects as go

In [60]:
# akses tabel persebaran corona di Indonesia dengan web scraping 
url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Indonesia'
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')
table = soup.find(class_ = 'wikitable float sortable').find('tbody')
table = table.find_all('tr')[2:-1]

In [61]:
# data persebaran corona di setiap provinsi di Indonesia
data = []
for row in table[:-1]:
    row_data = []
    province = row.find('th').text
    province = province.replace('\n', '')
    row_data.append(province)
    for i in row.find_all('td'):
        txt = i.text 
        txt = txt.replace('\n', '')
        row_data.append(txt)
    data.append(row_data)
row_data = []

# total penduduk Indonesia yang positif korona, meninggal, sembuh
for i in table[-1].find_all('th'):
    txt = i.text
    txt = txt.replace('\n', '')
    row_data.append(txt)
data.append(row_data)

In [62]:
data

[['Aceh', '5', '0', '0', '5', 'covid19.acehprov.go.id'],
 ['Bali', '10', '0', '2', '8', 'infocorona.baliprov.go.id'],
 ['Bangka Belitung', '0', '0', '0', '0', ''],
 ['Banten', '106', '1', '4', '101', 'infocorona.bantenprov.go.id'],
 ['Bengkulu', '0', '0', '0', '0', ''],
 ['Central Java', '63', '0', '7', '56', 'corona.jatengprov.go.id'],
 ['Central Kalimantan', '7', '0', '0', '7', 'corona.kalteng.go.id'],
 ['Central Sulawesi', '2', '0', '0', '2', ''],
 ['East Java', '90', '11', '7', '72', 'infocovid19.jatimprov.go.id'],
 ['East Kalimantan', '17', '0', '0', '17', 'covid19.kaltimprov.go.id'],
 ['East Nusa Tenggara', '0', '0', '0', '0', ''],
 ['Gorontalo', '0', '0', '0', '0', ''],
 ['Jakarta[b]', '720', '45', '68', '607', 'corona.jakarta.go.id'],
 ['Jambi', '1', '0', '0', '1', ''],
 ['Lampung', '4', '0', '0', '4', 'geoportal.lampungprov.go.id/corona/'],
 ['Maluku', '1', '0', '0', '1', 'corona.malukuprov.go.id'],
 ['North Maluku', '1', '0', '0', '1', ''],
 ['North Kalimantan', '2', '0', '0'

In [67]:
# create dataframe dari hasil scraping
df = pd.DataFrame(data, columns = ['Province', 'Confirmed', 'Recovered', 'Deaths', 'Active', 'Official website'])
df

,Province,Confirmed,Recovered,Deaths,Active,Official website
0,Aceh,5,0,0,5,covid19.acehprov.go.id
1,Bali,10,0,2,8,infocorona.baliprov.go.id
2,Bangka Belitung,0,0,0,0,
3,Banten,106,1,4,101,infocorona.bantenprov.go.id
4,Bengkulu,0,0,0,0,
5,Central Java,63,0,7,56,corona.jatengprov.go.id
6,Central Kalimantan,7,0,0,7,corona.kalteng.go.id
7,Central Sulawesi,2,0,0,2,
8,East Java,90,11,7,72,infocovid19.jatimprov.go.id
9,East Kalimantan,17,0,0,17,covid19.kaltimprov.go.id


In [84]:
# Peta indonesia
peta = fo.Map(
    location = [-6.284739, 106.641661],
    tiles = 'OpenStreetMap',
    zoom_start = 15
)
peta

In [85]:
# request API untuk mendapatkan koordinat setiap provinsi
url = 'https://raw.githubusercontent.com/LintangWisesa/Indonesia-Covid19-Maps/master/data/gps_indonesia.json'
data = requests.get(url)
data = data.json()

In [88]:
# tandai setiap provinsi dengan data persebaran corona
for i in df.index[:-2]:
    fo.Marker(
        location = [data[i]['latitude'], data[i]['longitude']],
        popup = 'Confirmed: {}, Recovered: {}, Deaths: {}, Active: {}'.format(df.iloc[i, 1], df.iloc[i, 2], df.iloc[i, 3], df.iloc[i, 4]),
        tooltip = '<b>' + df.iloc[i, 0] + '</b>',
        icon = fo.Icon(color='red')
    ).add_to(peta)
    
peta

In [90]:
# Save peta ke dalamm html
peta.save('Soal3_Covid19.html')